In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'c:\\PEC-26\\MLOPS - Journey\\Projects\\End-to-End-Wine-Quality-Prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """Data Ingestion Config class"""
    root_dir: Path
    source_URL: str
    local_file_name: Path
    unzip_dir: Path

In [4]:
from wine_quality.constants import *
from wine_quality.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath: Path = CONFIG_FILE_PATH, params_filepath: Path = PARAMS_FILE_PATH, schema_filepath: Path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def getDataIngestionConfig(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_file_name=config.local_file_name,
            unzip_dir=config.unzip_dir
        )    

In [6]:
import os
import urllib.request as request
import zipfile
from wine_quality import logger
from wine_quality.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # Ensure the parent directory exists
        os.makedirs(os.path.dirname(self.config.local_file_name), exist_ok=True)

        if not os.path.exists(self.config.local_file_name):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_file_name
            )
            logger.info(f"File downloaded: {filename} \nWith following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_file_name))}")
            
    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file to the specified directory
        Function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_file_name, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
#Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.getDataIngestionConfig()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2025-01-09 11:40:25,908: INFO: common yaml file: config\config.yaml loaded successfully]
[2025-01-09 11:40:25,910: INFO: common yaml file: params.yaml loaded successfully]
[2025-01-09 11:40:25,912: INFO: common yaml file: schema.yaml loaded successfully]
[2025-01-09 11:40:25,914: INFO: common created directory at: artifacts]
[2025-01-09 11:40:25,915: INFO: common created directory at: artifacts/data_ingestion]
[2025-01-09 11:40:25,916: INFO: 700563378 File already exists of size: ~ 25 KB]
